In [1]:
#-*-coding:utf-8-*-
import util
import tensorflow as tf
MNIST_CLASSIFIER_FROZEN_GRAPH = './classify_mnist_graph_def.pb'
INPUT_TENSOR = 'inputs:0'
OUTPUT_TENSOR = 'logits:0'
def EuclideanDistances(A, B):

    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(),ED.shape[0]*ED.shape[1])
def cal_distance_image_real(images,labels):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)
def cal_distance_image_fake(images):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    labels = tf.Session().run(tf.argmax(y_logits, 1))
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

    

In [2]:
import os
if not os.path.isdir('saved_models_mnist_{}'.format('sgan')):
    os.mkdir('saved_models_mnist_{}'.format('sgan'))
f = open('saved_models_mnist_{}/log_collapse1.txt'.format('sgan'), mode='w')
import os
import tensorflow as tf

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import losses
from keras.utils import to_categorical
import keras.backend as K

import matplotlib.pyplot as plt
import os
import numpy as np

class SGAN:
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(
            loss=['binary_crossentropy', 'categorical_crossentropy'],
            loss_weights=[0.5, 0.5],
            optimizer=optimizer,
            metrics=['accuracy']
        )

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        noise = Input(shape=(100,))
        img = self.generator(noise)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid, _ = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model(noise, valid)
        self.combined.compile(loss=['binary_crossentropy'], optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(1, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())

        model.summary()

        img = Input(shape=self.img_shape)

        features = model(img)
        valid = Dense(1, activation="sigmoid")(features)
        label = Dense(self.num_classes+1, activation="softmax")(features)

        return Model(img, [valid, label])

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (X_test, y_test) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        # X_test = X_test / 127.5 - 1.
        X_test = np.expand_dims(X_test, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Class weights:
        # To balance the difference in occurences of digit class labels.
        # 50% of labels that the discriminator trains on are 'fake'.
        # Weight = 1 / frequency
        half_batch = batch_size // 2
        cw1 = {0: 1, 1: 1}
        cw2 = {i: self.num_classes / half_batch for i in range(self.num_classes)}
        cw2[self.num_classes] = 1 / half_batch

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                # Sample noise and generate a batch of new images
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                gen_imgs = self.generator.predict(noise)

                # One-hot encoding of labels
                labels = to_categorical(y_train[index * batch_size:(index + 1) * batch_size], num_classes=self.num_classes + 1)
                fake_labels = to_categorical(np.full((batch_size, 1), self.num_classes),
                                             num_classes=self.num_classes + 1)

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(imgs, [valid, labels], class_weight=[cw1, cw2])
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, [fake, fake_labels], class_weight=[cw1, cw2])
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                g_loss = self.combined.train_on_batch(noise, valid, class_weight=[cw1, cw2])

                # Plot the progress
                print("epoch:%d step:%d[D loss: %f, acc: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (
                epoch,global_step, d_loss[0], 100 * d_loss[3], 100 * d_loss[4], g_loss))

                sampleSize = 5000
                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    s = self.sample_images(global_step, X_test,y_test, sampleSize)
    def sample_images(self, epoch,x_test,y_test,sample_num):
        r, c = sample_num//10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
#         sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('epoch:' + str(epoch))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')
if __name__ == '__main__':
    sgan = SGAN()
    sgan.train(epochs=30, batch_size=64, sample_interval=200)

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 64)          0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1[D loss: 0.622369, acc: 44.53%, op_acc: 3.91%] [G loss: 0.612243]
epoch:0 step:2[D loss: 0.545285, acc: 57.81%, op_acc: 7.03%] [G loss: 0.732943]
epoch:0 step:3[D loss: 0.489358, acc: 71.09%, op_acc: 9.38%] [G loss: 1.033745]
epoch:0 step:4[D loss: 0.421856, acc: 81.25%, op_acc: 10.94%] [G loss: 1.313544]
epoch:0 step:5[D loss: 0.362546, acc: 89.84%, op_acc: 10.94%] [G loss: 1.407000]
epoch:0 step:6[D loss: 0.329925, acc: 94.53%, op_acc: 13.28%] [G loss: 1.773664]
epoch:0 step:7[D loss: 0.316816, acc: 94.53%, op_acc: 20.31%] [G loss: 1.752508]
epoch:0 step:8[D loss: 0.317123, acc: 96.09%, op_acc: 14.84%] [G loss: 1.770294]
epoch:0 step:9[D loss: 0.289724, acc: 97.66%, op_acc: 17.19%] [G loss: 2.001392]
epoch:0 step:10[D loss: 0.297987, acc: 94.53%, op_acc: 10.94%] [G loss: 2.104426]
epoch:0 step:11[D loss: 0.269808, acc: 96.88%, op_acc: 20.31%] [G loss: 2.315678]
epoch:0 step:12[D loss: 0.264354, acc: 98.44%, op_acc: 28.91%] [G loss: 2.281291]
epoch:0 step:13[D loss: 0.24

epoch:0 step:102[D loss: 0.548904, acc: 56.25%, op_acc: 46.09%] [G loss: 2.324726]
epoch:0 step:103[D loss: 0.314086, acc: 78.12%, op_acc: 53.91%] [G loss: 2.363172]
epoch:0 step:104[D loss: 0.351578, acc: 71.88%, op_acc: 52.34%] [G loss: 2.049447]
epoch:0 step:105[D loss: 0.329856, acc: 73.44%, op_acc: 50.78%] [G loss: 2.701787]
epoch:0 step:106[D loss: 0.598227, acc: 46.88%, op_acc: 46.09%] [G loss: 1.931692]
epoch:0 step:107[D loss: 0.579703, acc: 54.69%, op_acc: 39.84%] [G loss: 1.910831]
epoch:0 step:108[D loss: 1.071783, acc: 20.31%, op_acc: 35.16%] [G loss: 1.640955]
epoch:0 step:109[D loss: 0.598943, acc: 49.22%, op_acc: 34.38%] [G loss: 1.706255]
epoch:0 step:110[D loss: 0.496779, acc: 55.47%, op_acc: 40.62%] [G loss: 1.946249]
epoch:0 step:111[D loss: 0.569629, acc: 47.66%, op_acc: 42.19%] [G loss: 1.539727]
epoch:0 step:112[D loss: 0.604426, acc: 43.75%, op_acc: 43.75%] [G loss: 1.915197]
epoch:0 step:113[D loss: 0.591251, acc: 49.22%, op_acc: 42.97%] [G loss: 2.009098]
epoc

...

epoch:29 step:27636[D loss: 0.336938, acc: 67.97%, op_acc: 53.91%] [G loss: 2.156835]
epoch:29 step:27637[D loss: 0.169790, acc: 91.41%, op_acc: 59.38%] [G loss: 1.350209]
epoch:29 step:27638[D loss: 0.194173, acc: 82.81%, op_acc: 71.09%] [G loss: 2.534787]
epoch:29 step:27639[D loss: 0.235033, acc: 78.91%, op_acc: 55.47%] [G loss: 2.124388]
epoch:29 step:27640[D loss: 0.131476, acc: 95.31%, op_acc: 71.09%] [G loss: 2.736834]
epoch:29 step:27641[D loss: 0.058057, acc: 100.00%, op_acc: 74.22%] [G loss: 2.515699]
epoch:29 step:27642[D loss: 0.046907, acc: 100.00%, op_acc: 78.12%] [G loss: 2.893094]
epoch:29 step:27643[D loss: 0.096173, acc: 96.09%, op_acc: 66.41%] [G loss: 1.741214]
epoch:29 step:27644[D loss: 0.012978, acc: 100.00%, op_acc: 92.97%] [G loss: 3.510310]
epoch:29 step:27645[D loss: 0.088509, acc: 98.44%, op_acc: 87.50%] [G loss: 2.455354]
epoch:29 step:27646[D loss: 0.822480, acc: 36.72%, op_acc: 42.97%] [G loss: 1.779741]
epoch:29 step:27647[D loss: 0.282519, acc: 73.44%, 

epoch:29 step:27736[D loss: 0.264843, acc: 75.78%, op_acc: 57.81%] [G loss: 1.467480]
epoch:29 step:27737[D loss: 0.286377, acc: 74.22%, op_acc: 62.50%] [G loss: 1.757138]
epoch:29 step:27738[D loss: 0.495789, acc: 46.09%, op_acc: 53.12%] [G loss: 1.682043]
epoch:29 step:27739[D loss: 0.286223, acc: 74.22%, op_acc: 57.81%] [G loss: 1.175572]
epoch:29 step:27740[D loss: 0.136397, acc: 91.41%, op_acc: 62.50%] [G loss: 2.226440]
epoch:29 step:27741[D loss: 0.484685, acc: 51.56%, op_acc: 60.16%] [G loss: 1.185457]
epoch:29 step:27742[D loss: 0.400774, acc: 50.78%, op_acc: 52.34%] [G loss: 1.239950]
epoch:29 step:27743[D loss: 0.199174, acc: 87.50%, op_acc: 57.03%] [G loss: 1.288592]
epoch:29 step:27744[D loss: 0.207097, acc: 89.84%, op_acc: 67.19%] [G loss: 1.534418]
epoch:29 step:27745[D loss: 0.353638, acc: 60.16%, op_acc: 73.44%] [G loss: 1.123658]
epoch:29 step:27746[D loss: 0.396442, acc: 54.69%, op_acc: 56.25%] [G loss: 0.865118]
epoch:29 step:27747[D loss: 0.333877, acc: 67.97%, op_

epoch:29 step:27830[D loss: 0.368132, acc: 60.16%, op_acc: 57.81%] [G loss: 0.867123]
epoch:29 step:27831[D loss: 0.200849, acc: 89.84%, op_acc: 64.84%] [G loss: 1.752108]
epoch:29 step:27832[D loss: 0.260830, acc: 82.03%, op_acc: 61.72%] [G loss: 1.224911]
epoch:29 step:27833[D loss: 0.340123, acc: 67.97%, op_acc: 60.94%] [G loss: 1.172422]
epoch:29 step:27834[D loss: 0.292993, acc: 71.88%, op_acc: 58.59%] [G loss: 1.426673]
epoch:29 step:27835[D loss: 0.212009, acc: 84.38%, op_acc: 60.94%] [G loss: 2.005616]
epoch:29 step:27836[D loss: 0.075857, acc: 98.44%, op_acc: 71.09%] [G loss: 2.571312]
epoch:29 step:27837[D loss: 0.050120, acc: 100.00%, op_acc: 80.47%] [G loss: 2.102602]
epoch:29 step:27838[D loss: 0.057036, acc: 100.00%, op_acc: 82.03%] [G loss: 2.574873]
epoch:29 step:27839[D loss: 0.123810, acc: 95.31%, op_acc: 67.97%] [G loss: 2.496787]
epoch:29 step:27840[D loss: 0.106140, acc: 99.22%, op_acc: 79.69%] [G loss: 2.531229]
epoch:29 step:27841[D loss: 0.637736, acc: 23.44%, o

epoch:29 step:27930[D loss: 0.206067, acc: 83.59%, op_acc: 60.94%] [G loss: 1.379644]
epoch:29 step:27931[D loss: 0.295287, acc: 72.66%, op_acc: 65.62%] [G loss: 1.628999]
epoch:29 step:27932[D loss: 0.835265, acc: 28.91%, op_acc: 34.38%] [G loss: 1.692585]
epoch:29 step:27933[D loss: 0.518447, acc: 45.31%, op_acc: 57.03%] [G loss: 1.416607]
epoch:29 step:27934[D loss: 0.403794, acc: 53.91%, op_acc: 51.56%] [G loss: 0.768820]
epoch:29 step:27935[D loss: 0.215599, acc: 85.16%, op_acc: 57.81%] [G loss: 1.710488]
epoch:29 step:27936[D loss: 0.345466, acc: 65.62%, op_acc: 50.78%] [G loss: 1.203151]
epoch:29 step:27937[D loss: 0.065667, acc: 100.00%, op_acc: 78.12%] [G loss: 2.945099]
epoch:29 step:27938[D loss: 0.621551, acc: 40.62%, op_acc: 50.00%] [G loss: 1.935531]
epoch:29 step:27939[D loss: 0.331331, acc: 67.19%, op_acc: 66.41%] [G loss: 2.302294]
epoch:29 step:27940[D loss: 0.245142, acc: 81.25%, op_acc: 57.81%] [G loss: 1.610322]
epoch:29 step:27941[D loss: 0.125639, acc: 95.31%, op

epoch:29 step:28027[D loss: 0.069566, acc: 97.66%, op_acc: 92.97%] [G loss: 2.772873]
epoch:29 step:28028[D loss: 0.323158, acc: 64.84%, op_acc: 75.78%] [G loss: 2.072842]
epoch:29 step:28029[D loss: 0.118871, acc: 100.00%, op_acc: 74.22%] [G loss: 1.297971]
epoch:29 step:28030[D loss: 0.134035, acc: 92.19%, op_acc: 67.19%] [G loss: 1.286463]
epoch:29 step:28031[D loss: 0.135561, acc: 96.09%, op_acc: 66.41%] [G loss: 1.216668]
epoch:29 step:28032[D loss: 0.154632, acc: 94.53%, op_acc: 70.31%] [G loss: 1.719138]
epoch:29 step:28033[D loss: 0.140690, acc: 96.09%, op_acc: 75.78%] [G loss: 2.126060]
epoch:29 step:28034[D loss: 0.167000, acc: 94.53%, op_acc: 66.41%] [G loss: 1.802677]
epoch:29 step:28035[D loss: 0.294994, acc: 72.66%, op_acc: 69.53%] [G loss: 1.337925]
epoch:29 step:28036[D loss: 0.452864, acc: 50.00%, op_acc: 53.91%] [G loss: 1.649228]
epoch:29 step:28037[D loss: 0.411299, acc: 51.56%, op_acc: 57.81%] [G loss: 1.013009]
epoch:29 step:28038[D loss: 0.314273, acc: 69.53%, op